In [1]:
import pandas as pd

In [3]:
match_df = pd.read_csv('./datasets/IPL_Matches_2008_2022.csv')
match_df.head()

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,N,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,N,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,N,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,N,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan


In [4]:
ball_by_ball_df = pd.read_csv('./datasets/IPL_Ball_by_Ball_2008_2022.csv')
ball_by_ball_df.head()

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals


In [5]:
motm = match_df['Player_of_Match'].value_counts()
motm

Player_of_Match
AB de Villiers    25
CH Gayle          22
DA Warner         18
RG Sharma         18
MS Dhoni          17
                  ..
CR Brathwaite      1
Mohsin Khan        1
A Zampa            1
BCJ Cutting        1
MF Maharoof        1
Name: count, Length: 262, dtype: int64

In [64]:
batter_data = ball_by_ball_df[['ID', 'batter', 'batsman_run', 'isWicketDelivery', 'total_run']]

batter_agg = batter_data.groupby('batter').agg(
    matches=('ID', 'nunique'),
    total_runs=('batsman_run', 'sum'),
    total_balls=('batsman_run', 'count'),
    outs=('isWicketDelivery', 'sum')
).reset_index()

batter_agg

,batter,innings,total_runs,total_balls,outs
0,A Ashish Reddy,23,280,196,15
1,A Badoni,11,161,139,9
2,A Chandila,2,4,7,1
3,A Chopra,6,53,75,5
4,A Choudhary,3,25,20,2
...,...,...,...,...,...
600,Yash Dayal,1,0,1,1
601,Yashpal Singh,4,47,67,4
602,Younis Khan,1,3,7,1
603,Yuvraj Singh,126,2754,2207,109


In [65]:
runs_per_match = batter_data.groupby(['ID', 'batter'])['batsman_run'].sum().reset_index()
runs_per_match['50s'] = (runs_per_match['batsman_run'] >= 50) & (runs_per_match['batsman_run'] < 100)
runs_per_match['100s'] = runs_per_match['batsman_run'] >= 100

balls_faced_per_match = batter_data.groupby(['ID', 'batter'])['batsman_run'].count().reset_index()
balls_faced_per_match.columns = ['ID', 'batter', 'balls_faced']
runs_per_match = pd.merge(runs_per_match, balls_faced_per_match, on=['ID', 'batter'], how='left')

runs_per_match['strike_rate'] = runs_per_match['batsman_run'] / runs_per_match['balls_faced'] * 100

runs_per_match

,ID,batter,batsman_run,50s,100s,balls_faced,strike_rate
0,335982,AA Noffke,9,False,False,12,75.000000
1,335982,B Akhil,0,False,False,2,0.000000
2,335982,BB McCullum,158,False,True,77,205.194805
3,335982,CL White,6,False,False,10,60.000000
4,335982,DJ Hussey,12,False,False,12,100.000000
...,...,...,...,...,...,...,...
14224,1312200,SV Samson,14,False,False,11,127.272727
14225,1312200,Shubman Gill,45,False,False,43,104.651163
14226,1312200,TA Boult,11,False,False,7,157.142857
14227,1312200,WP Saha,5,False,False,7,71.428571


In [66]:
batter_50s_100s = runs_per_match.groupby('batter').agg(
    total_50s=('50s', 'sum'),
    total_100s=('100s', 'sum'),
    avg_strike_rate=('strike_rate', 'mean')
).reset_index()

batter_50s_100s

,batter,total_50s,total_100s,avg_strike_rate
0,A Ashish Reddy,0,0,135.390395
1,A Badoni,1,0,117.098930
2,A Chandila,0,0,33.333333
3,A Chopra,0,0,59.148148
4,A Choudhary,0,0,163.888889
...,...,...,...,...
600,Yash Dayal,0,0,0.000000
601,Yashpal Singh,0,0,75.753968
602,Younis Khan,0,0,42.857143
603,Yuvraj Singh,13,0,110.808778


In [67]:
batter_agg = pd.merge(batter_agg, batter_50s_100s, on='batter', how='left')

batter_agg['motm_count'] = batter_agg['batter'].map(motm).fillna(0).astype(int)

In [68]:
batter_agg.head()

,batter,innings,total_runs,total_balls,outs,total_50s,total_100s,avg_strike_rate,motm_count
0,A Ashish Reddy,23,280,196,15,0,0,135.390395,0
1,A Badoni,11,161,139,9,1,0,117.098930,0
2,A Chandila,2,4,7,1,0,0,33.333333,1
3,A Chopra,6,53,75,5,0,0,59.148148,0
4,A Choudhary,3,25,20,2,0,0,163.888889,0


In [69]:
batter_agg.dtypes

batter              object
innings              int64
total_runs           int64
total_balls          int64
outs                 int64
total_50s            int64
total_100s           int64
avg_strike_rate    float64
motm_count           int64
dtype: object

In [71]:
# adding total orange caps
batter_agg[batter_agg['batter'].str.contains('Kishan')]

,batter,innings,total_runs,total_balls,outs,total_50s,total_100s,avg_strike_rate,motm_count
209,Ishan Kishan,70,1870,1444,62,12,0,110.947435,3


In [72]:
batter_agg.loc[batter_agg['batter'].str.contains('SE Marsh'), 'orange_caps'] = 1
batter_agg.loc[batter_agg['batter'].str.contains('ML Hayden'), 'orange_caps'] = 1
batter_agg.loc[batter_agg['batter'].str.contains('SR Tendulkar'), 'orange_caps'] = 1
batter_agg.loc[batter_agg['batter'].str.contains('CH Gayle'), 'orange_caps'] = 2
batter_agg.loc[batter_agg['batter'].str.contains('MEK Hussey'), 'orange_caps'] = 1
batter_agg.loc[batter_agg['batter'].str.contains('RV Uthappa'), 'orange_caps'] = 1
batter_agg.loc[batter_agg['batter'].str.contains('DA Warner'), 'orange_caps'] = 3
batter_agg.loc[batter_agg['batter'].str.contains('V Kohli'), 'orange_caps'] = 1
batter_agg.loc[batter_agg['batter'].str.contains('KS Williamson'), 'orange_caps'] = 1
batter_agg.loc[batter_agg['batter'].str.contains('KL Rahul'), 'orange_caps'] = 1
batter_agg.loc[batter_agg['batter'].str.contains('RD Gaikwad'), 'orange_caps'] = 1
batter_agg.loc[batter_agg['batter'].str.contains('JC Buttler'), 'orange_caps'] = 1
batter_agg['orange_caps'] = batter_agg['orange_caps'].fillna(0).astype(int)

In [73]:
batter_agg['average'] = batter_agg['total_runs'] / batter_agg['innings']

In [74]:
fours_sixes_data = ball_by_ball_df[ball_by_ball_df['batsman_run'].isin([4, 6]) & (ball_by_ball_df['non_boundary'] == 0)]

fours_sixes_agg = fours_sixes_data.groupby(['batter', 'batsman_run']).size().unstack(fill_value=0).reset_index()

# Renaming the columns for clarity
fours_sixes_agg.columns = ['batter', 'fours', 'sixes']

fours_sixes_agg

,batter,fours,sixes
0,A Ashish Reddy,16,15
1,A Badoni,11,7
2,A Chopra,7,0
3,A Choudhary,1,1
4,A Flintoff,5,2
...,...,...,...
498,YK Pathan,263,160
499,YV Takawale,26,3
500,Yashpal Singh,5,0
501,Yuvraj Singh,218,149


In [75]:
batter_agg = pd.merge(batter_agg, fours_sixes_agg, on='batter', how='left')

In [76]:
batter_agg['fours'] = batter_agg['fours'].fillna(0).astype(int)
batter_agg['sixes'] = batter_agg['sixes'].fillna(0).astype(int)

batter_agg

,batter,innings,total_runs,total_balls,outs,total_50s,total_100s,avg_strike_rate,motm_count,orange_caps,average,fours,sixes
0,A Ashish Reddy,23,280,196,15,0,0,135.390395,0,0,12.173913,16,15
1,A Badoni,11,161,139,9,1,0,117.098930,0,0,14.636364,11,7
2,A Chandila,2,4,7,1,0,0,33.333333,1,0,2.000000,0,0
3,A Chopra,6,53,75,5,0,0,59.148148,0,0,8.833333,7,0
4,A Choudhary,3,25,20,2,0,0,163.888889,0,0,8.333333,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,Yash Dayal,1,0,1,1,0,0,0.000000,0,0,0.000000,0,0
601,Yashpal Singh,4,47,67,4,0,0,75.753968,0,0,11.750000,5,0
602,Younis Khan,1,3,7,1,0,0,42.857143,0,0,3.000000,0,0
603,Yuvraj Singh,126,2754,2207,109,13,0,110.808778,5,0,21.857143,218,149


In [77]:
batter_agg.isna().sum()

batter             0
innings            0
total_runs         0
total_balls        0
outs               0
total_50s          0
total_100s         0
avg_strike_rate    0
motm_count         0
orange_caps        0
average            0
fours              0
sixes              0
dtype: int64

In [78]:
batter_agg.to_csv('./datasets/batter_agg.csv', index=False)